# 1. Import Library

In [42]:
import os
import pandas as pd
import difflib
from collections import defaultdict
import csv
import matplotlib.pyplot as plt


# 2. Load Dataset

In [44]:
PROVINCE_NAMES = [
    'Hà Giang', 'Cao Bằng', 'Lạng Sơn', 'Bắc Giang', 'Phú Thọ', 'Thái Nguyên', 'Bắc Kạn', 'Tuyên Quang', 
    'Lào Cai', 'Yên Bái', 'Lai Châu', 'Sơn La', 'Điện Biên', 'Hòa Bình', 'Hà Nội', 'Hải Phòng', 'Hải Dương', 
    'Hưng Yên', 'Vĩnh Phúc', 'Bắc Ninh', 'Thái Bình', 'Nam Định', 'Hà Nam', 'Ninh Bình', 'Quảng Ninh', 
    'Thanh Hóa', 'Nghệ An', 'Hà Tĩnh', 'Quảng Bình', 'Quảng Trị', 'Thừa Thiên - Huế', 'Đà Nẵng', 'Quảng Nam', 
    'Quảng Ngãi', 'Bình Định', 'Phú Yên', 'Khánh Hòa', 'Ninh Thuận', 'Bình Thuận', 'Kon Tum', 'Gia Lai', 
    'Đắk Lắk', 'Đắk Nông', 'Lâm Đồng', 'TP. Hồ Chí Minh', 'Đồng Nai', 'Bà Rịa - Vũng Tàu', 'Bình Dương', 
    'Bình Phước', 'Tây Ninh', 'TP. Cần Thơ', 'Long An', 'Tiền Giang', 'Bến Tre', 'Trà Vinh', 'Vĩnh Long', 
    'An Giang', 'Đồng Tháp', 'Kiên Giang', 'Hậu Giang', 'Sóc Trăng', 'Bạc Liêu', 'Cà Mau'
]

In [59]:
YEARS = [2018, 2019, 2020, 2021, 2022, 2023]

In [118]:
path_university_score = r"DATA\CLEAN DATA\Điểm thi THPTQG [2018-2023]\university_scores_clean.csv"
path_education_investment = r"DATA\CLEAN DATA\Đầu tư giáo dục [2018-2023]\clean_education_data.csv"
path_population_density = r"DATA\CLEAN DATA\Dân số và mật độ dân số [2018-2022]\clean_population.csv"
path_income = r"DATA\CLEAN DATA\Chỉ số thu nhập [2018-2020]\Chỉ số thu nhập [2018-2020].csv"
path_health_index = r"DATA\CLEAN DATA\Chỉ số sức khỏe [2018-2020]\Chỉ số sức khỏe [2018-2020].csv"
path_human_development_index = r"DATA\CLEAN DATA\Chỉ số phát triển con người [2018-2020]\Chỉ số phát triển con người [2018-2020].csv"
path_education_index = r"DATA\CLEAN DATA\Chỉ số giáo dục [2018-2020]\Chỉ số giáo dục [2018-2020].csv"
path_economic_regions = r"DATA\CLEAN DATA\6 vùng kinh tế\regions_data.csv"

# List các đường dẫn
paths = [
    path_university_score,
    path_education_investment,
    path_population_density,
    path_income,
    path_health_index,
    path_human_development_index,
    path_education_index,
    path_economic_regions
]

In [50]:
for path in paths:
    print("File:", path)
    try:
        df = pd.read_csv(path)
        print(df.info())  # In ra thông tin về cấu trúc của DataFrame
        print("Số lượng giá trị null trong từng cột:")
        print(df.isnull().sum())  # In ra số lượng giá trị null trong mỗi cột
        print("\n")
    except FileNotFoundError:
        print("File not found.")

File: DATA\CLEAN DATA\Điểm thi THPTQG [2018-2023]\university_scores_clean.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Địa phương  378 non-null    object 
 1   Năm         378 non-null    int64  
 2   Giá trị     378 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.0+ KB
None
Số lượng giá trị null trong từng cột:
Địa phương    0
Năm           0
Giá trị       0
dtype: int64


File: DATA\CLEAN DATA\Đầu tư giáo dục [2018-2023]\clean_education_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Địa phương             295 non-null    object 
 1   Năm                    295 non-null    int64  
 2   Chi tiêu cho giáo dục  295 non-null    float64
dtypes

# 3.Finding Multivariate Relationships

In [124]:
data = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))

## 3.1 Process University Scores

In [125]:
df = pd.read_csv(path_university_score)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Giá trị')

for province in PROVINCE_NAMES:
    for year in YEARS:
        data[province][year]["university_score"] = df_pivot[province][year]

print(df_pivot['Đà Nẵng'][2023])
print(data['Đà Nẵng'][2023]['university_score'])


6.265920651068158
6.265920651068158


## 3.2 Process Economic Zone

In [126]:
df = pd.read_csv(path_economic_regions)

for province in PROVINCE_NAMES:
    for year in YEARS:
        # Lọc dữ liệu cho tỉnh và in ra cột "Vùng kinh tế - Xã hội"
        data[province][year]["economic_zone"] = df[df["Địa phương"] == province]["Vùng kinh tế - Xã hội"].iloc[0]

print(df[df["Địa phương"] == "Đà Nẵng"]['Vùng kinh tế - Xã hội'].iloc[0])
print(data["Đà Nẵng"][2018]["economic_zone"])


Vùng Bắc Trung Bộ và duyên hải miền Trung
Vùng Bắc Trung Bộ và duyên hải miền Trung


## 3.3 Process Education Investment

In [127]:
df = pd.read_csv(path_education_investment)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Chi tiêu cho giáo dục')

for province in PROVINCE_NAMES:
    for year in YEARS:
        if province in df_pivot.columns and year in df_pivot.index:
            data[province][year]["education_investment"] = df_pivot[province][year]

print(df_pivot['An Giang'][2018])
print(data['An Giang'][2018]['education_investment'])

0.698327
0.698327


## 3.4 Process Population And Population Density

In [129]:
import pandas as pd

# Đọc dữ liệu từ file CSV
df = pd.read_csv(path_population_density)

for province in PROVINCE_NAMES:
    for year in YEARS:
        # Kiểm tra xem có tồn tại hàng thỏa mãn điều kiện 'Địa phương' và 'Năm' không
        if ((df['Địa phương'] == province) & (df['Năm'] == year)).any():
            # Lấy dữ liệu từ DataFrame cho province và year tương ứng
            popu_data = df[(df['Địa phương'] == province) & (df['Năm'] == year)]
            population = popu_data['Dân số trung bình (Nghìn người)'].values[0]
            population_density = popu_data['Mật độ dân số (Người/km2)'].values[0]
            data[province][year]["population"] = population
            data[province][year]["population_density"] = population_density

print(df[(df['Địa phương'] == 'Hà Nội') & (df['Năm'] == 2018)]['Dân số trung bình (Nghìn người)'].values[0], df[(df['Địa phương'] == 'Hà Nội') & (df['Năm'] == 2018)]['Mật độ dân số (Người/km2)'].values[0])
print(data["Hà Nội"][2018]['population'], data["Hà Nội"][2018]['population_density'])


7914.5 2356.49
7914.5 2356.49


## 3.5 Process Educational Index

In [134]:
df = pd.read_csv(path_education_index)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Giá trị')

for province in PROVINCE_NAMES:
    for year in range(2018, 2021):
        data[province][year]["education_index"] = df_pivot[province][year]

print(df_pivot['Đà Nẵng'][2018])
print(data['Đà Nẵng'][2018]['education_index'])


0,745
0,745


## 3.6 Process Human Development Index

In [135]:
df = pd.read_csv(path_human_development_index)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Giá trị')

for province in PROVINCE_NAMES:
    for year in range(2018, 2021):
        data[province][year]["human_development_index"] = df_pivot[province][year]

print(df_pivot['Đà Nẵng'][2018])
print(data['Đà Nẵng'][2018]['human_development_index'])


0,773
0,773


## 3.7 Process Health Index

In [136]:
df = pd.read_csv(path_health_index)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Giá trị')

for province in PROVINCE_NAMES:
    for year in range(2018, 2021):
        data[province][year]["health_index"] = df_pivot[province][year]

print(df_pivot['Đà Nẵng'][2018])
print(data['Đà Nẵng'][2018]['health_index'])


0,862
0,862


## 3.8 Process Income Index

In [137]:
df = pd.read_csv(path_income)

# Pivot DataFrame để có các tỉnh thành làm cột
df_pivot = df.pivot(index='Năm', columns='Địa phương', values='Giá trị')

for province in PROVINCE_NAMES:
    for year in range(2018, 2021):
        data[province][year]["income_index"] = df_pivot[province][year]

print(df_pivot['Đà Nẵng'][2018])
print(data['Đà Nẵng'][2018]['income_index'])


0,719
0,719


In [139]:
print(data["Đà Nẵng"][2018]['education_index'])

0,745


In [142]:

# Đường dẫn đến file CSV
csv_file = 'output.csv'

# Ghi dữ liệu vào file CSV
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Viết header
    writer.writerow(['Địa phương', 'Năm', 'Vùng kinh tế', 'Tiền đầu tư giáo dục', 
                     'Dân số trung bình', 'Mật độ dân số trung bình', 'Chỉ số Giáo dục',
                     'Chỉ số phát triển con người', 'Chỉ số sức khỏe', 'Chỉ số thu nhập', 
                     'Điểm thi đại học trung bình của địa phương'])
    
    # Viết dữ liệu từ defaultdict vào file CSV
    for province, years_data in data.items():
        for year, values in years_data.items():
            writer.writerow([province, year, values['Vùng kinh tế'], values['Tiền đầu tư giáo dục'],
                             values['Dân số trung bình'], values['Mật độ dân số trung bình'],
                             values['Chỉ số Giáo dục'], values['Chỉ số phát triển con người'],
                             values['Chỉ số sức khỏe'], values['Chỉ số thu nhập'],
                             values['Điểm thi đại học trung bình của địa phương']])

dict_keys(['Hà Giang', 'Cao Bằng', 'Lạng Sơn', 'Bắc Giang', 'Phú Thọ', 'Thái Nguyên', 'Bắc Kạn', 'Tuyên Quang', 'Lào Cai', 'Yên Bái', 'Lai Châu', 'Sơn La', 'Điện Biên', 'Hòa Bình', 'Hà Nội', 'Hải Phòng', 'Hải Dương', 'Hưng Yên', 'Vĩnh Phúc', 'Bắc Ninh', 'Thái Bình', 'Nam Định', 'Hà Nam', 'Ninh Bình', 'Quảng Ninh', 'Thanh Hóa', 'Nghệ An', 'Hà Tĩnh', 'Quảng Bình', 'Quảng Trị', 'Thừa Thiên - Huế', 'Đà Nẵng', 'Quảng Nam', 'Quảng Ngãi', 'Bình Định', 'Phú Yên', 'Khánh Hòa', 'Ninh Thuận', 'Bình Thuận', 'Kon Tum', 'Gia Lai', 'Đắk Lắk', 'Đắk Nông', 'Lâm Đồng', 'TP. Hồ Chí Minh', 'Đồng Nai', 'Bà Rịa - Vũng Tàu', 'Bình Dương', 'Bình Phước', 'Tây Ninh', 'TP. Cần Thơ', 'Long An', 'Tiền Giang', 'Bến Tre', 'Trà Vinh', 'Vĩnh Long', 'An Giang', 'Đồng Tháp', 'Kiên Giang', 'Hậu Giang', 'Sóc Trăng', 'Bạc Liêu', 'Cà Mau', 0])


TypeError: unhashable type: 'collections.defaultdict'